## Summary

"Unsaved report
"October 1, 2018 - September 30, 2019"
"

The following data set if for an ecommerce company that sells tires.  The data set is from Google paid search queries related to Product Listing Ads (PLAs) which include Google Shopping queries. The search terms that include Client’s brand name have been excluded. Given this data, and any other information you want to add, what can you tell us? What information/visualizations can we share with the customer to help them position/re-position etc. How are people finding their products, and what can we do with this information?

Original Query Parameters:

Campaign Type: Google Shopping PLAs
Network: Google Search; Excluding all other Audience Extension Networks
Note: This file is based upon Search Terms (not Keywords), which are the actual queries that users typed in

Conversion Parameters (e.g., how is Conversions and Conversion Value calculated):

Includes the revenue generated from transactions that occur within 60 days of an ad click; multiple transactions within 60 days are counted
Conversion Values are modeled on a “Position-based attribution model”, which means that Google Ads allocates conversion value to ads based on: First Ad Click = 40%, Last Ad Click = 40%, All Middle Clicks = 20% (Linear Split)
As a result, Conversions and Conversion Values are not a perfect measure as we don’t know if the conversion value was due to a search query being the 1st click, last click, or a middle click

Calculations Not Included:
CV Gross Profit = Conversion Value * 20%
CV Gross Profit Less Ad Cost = (Conversion Value * 20% ) – Ad Cost
MCF Revenue = (Conversion Value * 50%)
MCF Gross Profit = ((Conversion Value * 50%) *20%) - https://www.ppchero.com/ultimate-guide-to-googles-multi-channel-funnels/
MCF Gross Profit Less Ad Cost = ((Conversion Value * 50%) * 20%) – Ad Cost

 - Impression: https://www.klipfolio.com/metrics/difference/ad-clicks-vs-ad-impressions#:~:text=Ad%20Clicks%2C%20or%20simply%20Clicks,to%20reach%20an%20online%20property.&text=Ad%20Impressions%20(IMPR)%20is%20a,screen%20within%20the%20publisher's%20network.
 
 - Impression Share: https://support.google.com/google-ads/answer/2497703?hl=en
  
 - Conversion: A conversion in Google Ads is when a user performs some specified action after clicking an ad or viewing a Display Network ad, such as purchasing a product, installing a mobile app, or signing up for an email list. Conversion tracking provides key insights into users' actions after viewing or clicking an ad.

## Imports & Such

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random as ran
%matplotlib inline
import re

In [2]:
# Imports
# Natural Language Processing libraries, initiations and functions
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import re


# Preprocessing
def preprocess(text):
    text = re.sub(r'[^a-zA-Z]',' ', text.lower())
    tokens = word_tokenize(text)
    lemmer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    return " ".join([lemmer.lemmatize(word) for word 
                     in tokens if len(word) > 3 and not word in stop_words])



# Cvec, Standard
cvec = CountVectorizer(analyzer = "word",
                       min_df = 2,
                       preprocessor = preprocess,
                       stop_words = 'english') 
# Cvec DF
#df_words = pd.DataFrame(cvec.fit_transform(df['doc_column']).todense(), 
#                        columns=cvec.get_feature_names())

In [3]:
df = pd.read_csv('./data/temp_clean.csv', index_col= False)
df_raw = pd.read_csv('./data/Search_terms_cleaned.csv')#.columns

C:\Users\Peridot of Earth\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.head()

,Unnamed: 0,Search_term,Currency,Cost,Avg_cpc,Impressions,Clicks,Impr_top_%,Impr_abs_top_%,Conversions,Conv_value,Conv_rate,Conv_value_cost,CV_gross_profit,CV_gross_profit_less_ad_cost,MCF_revenue,MCF_gross_profit,MCF_gross_profit_less_ad_cost
0,0,285 60r20 sale,USD,2.22,0.74,3,3,0.0,0.0,0.00,0.00,0.00,0.00,0.000,-2.220,0.00,0.000,-2.220
1,1,325 45r22 nitto 420s,USD,0.22,0.22,1,1,0.0,0.0,0.00,0.00,0.00,0.00,0.000,-0.220,0.00,0.000,-0.220
2,2,wild country at,USD,3.78,0.22,39,17,0.0,0.0,0.07,6.32,0.43,1.67,1.264,-2.516,3.16,0.632,-3.148
3,3,h w 100xt primex tires 7.50 16,USD,0.17,0.17,1,1,0.0,0.0,0.00,0.00,0.00,0.00,0.000,-0.170,0.00,0.000,-0.170
4,4,285 75r16 at tires,USD,21.69,0.39,263,56,0.0,0.0,0.63,438.10,1.10,20.20,87.620,65.930,219.05,43.810,22.120


## Cleaning

In [5]:
df.drop(['Unnamed: 0', 'Currency', 'Impr_top_%', 'Impr_abs_top_%','CV_gross_profit', 'CV_gross_profit_less_ad_cost', 
         'MCF_revenue', 'MCF_gross_profit', 'MCF_gross_profit_less_ad_cost'], axis = 1, inplace = True)

## Exploritory Data Analysis

In [6]:
# Sum of all Columns
for col in df.drop('Search_term', axis = 1).columns:
    print("=================")
    print(col)
    print(np.round(df[col].sum()))

Cost
2876546.0
Avg_cpc
738352.0
Impressions
95671871
Clicks
7761976
Conversions
60783.0
Conv_value
23596698.0
Conv_rate
1779750.0
Conv_value_cost
64754878.0


In [7]:
# Average of All Columns
for col in df.drop('Search_term', axis = 1).columns:
    print("=================")
    print(col)
    print(np.average(df[col]))

Cost
1.278633886339663
Avg_cpc
0.32819972236335226
Impressions
42.52644504630167
Clicks
3.4502225404864553
Conversions
0.02701817528802689
Conv_value
10.488805935716842
Conv_rate
0.7911042702081124
Conv_value_cost
28.783745192143126


In [8]:
df.shape

(2249703, 9)

In [9]:
df.sort_values('Conv_value_cost', ascending= False).head(10)

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost
2205971,sku nito 217250,0.01,0.01,1,1,2.0,3640.00,200.0,364000.0
534102,yokohamamy507 255 70r22 5,0.01,0.01,1,1,1.0,2717.52,100.0,271752.0
642928,goodyear dyna torque r1 18.4 r42,0.01,0.01,1,1,1.0,2649.84,100.0,264984.0
724416,goodyear endurance rsa ult 245 70 19.5,0.01,0.01,1,1,1.0,1703.94,100.0,170394.0
269525,yokohama tires on sale on ebay,0.01,0.01,3,1,1.0,1583.88,100.0,158388.0
1376687,goodyear assurance cs tripletred as radial 245...,0.01,0.01,1,1,2.0,1547.36,200.0,154736.0
2088569,mu tires for truck radar,0.01,0.01,1,1,1.0,1459.64,100.0,145964.0
949490,goodyear wrangler duratrac 20 lt,0.01,0.01,1,1,1.0,1455.96,100.0,145596.0
1999892,p205 60r15 michelin,0.01,0.01,1,1,2.6,1371.81,260.0,137180.8
1212854,goodyear 11r 22.5 399 lhs,0.01,0.01,1,1,0.4,1334.14,40.0,133414.4


In [10]:
df.sort_values('Avg_cpc', ascending= False).head(10)

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost
598529,goodyear all season tires 245 50 20,23.20,23.20,1,1,0.0,0.0,0.0,0.0
1706964,nitto nt421 265 50 20,16.75,16.75,1,1,0.0,0.0,0.0,0.0
1428623,goodyear eagle all season reviews,15.61,15.61,1,1,0.0,0.0,0.0,0.0
148068,2157514,15.35,15.35,4,1,0.0,0.0,0.0,0.0
1660559,tires online 285 45r22,30.22,15.11,3,2,0.0,0.0,0.0,0.0
1252634,295 40 22 all season,14.53,14.53,1,1,0.0,0.0,0.0,0.0
2157933,tires easy 255 55r20,14.39,14.39,1,1,0.0,0.0,0.0,0.0
417088,atturo az800 275 60r20xl 119v bsw,14.13,14.13,1,1,0.0,0.0,0.0,0.0
1652028,hankook 205 45r17 88v runflat optimo,14.07,14.07,1,1,0.0,0.0,0.0,0.0
904699,2008 honda pilot wheels,13.95,13.95,1,1,0.0,0.0,0.0,0.0


In [11]:
df_singles = df[df['Clicks'] == 1]

In [12]:
list_results = []
for row in df['Search_term']:
    result = re.findall(r'[\d.rbyhpx ]+', row)
    
    list_inner = []
    for item in result:
        item = item.strip(' ')
        if len(item) > 7:
            list_inner.append(item)
    list_results.append(list_inner)

In [13]:
list_lists_count = []
for num in list_results:
    list_lists_count.append(len(num))

for num in range(11):
    print(num, list_lists_count.count(num))

0 883531
1 1355671
2 10399
3 88
4 12
5 0
6 2
7 0
8 0
9 0
10 0


In [14]:
df['list_dimension'] = list_results

In [15]:
df['list_counts'] = list_lists_count

In [16]:
df_measures = df[df['list_counts'] == 1]

In [17]:
# Preprocessing
def preprocess(text):
    text = re.sub(r'[^a-z]',' ', text)
    tokens = word_tokenize(text)
    lemmer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    return " ".join([lemmer.lemmatize(word) for word 
                     in tokens if len(word) > 5 and not word in stop_words])



# Cvec, Standard
cvec = CountVectorizer(analyzer = "word",
                       min_df = 5,
                       preprocessor = preprocess,
                       stop_words = 'english') 
# Cvec DF
df_words = pd.DataFrame(cvec.fit_transform(df_measures['Search_term']).todense(), 
                        columns=cvec.get_feature_names())

In [ ]:
df_words.to_csv('./data/cvec_with_but_without_numbers.csv')

In [19]:
counts = df_words.sum()

In [18]:
df_words.sum().sort_values(ascending=False).head(20)

terrain        53582
goodyear       46211
michelin       39832
cooper         39789
walmart        25084
firestone      23556
country        23487
radial         22770
season         22355
grappler       21329
hankook        21114
falken         19973
federal        18900
bridgestone    18068
goodrich       16460
wrangler       16057
yokohama       15924
continental    15724
mastercraft    15464
tractor        15416
dtype: int64

In [20]:
test = counts[counts >= 1000]
test.

accelera        2075
achilles        9536
advanta         1155
advantage       1285
adventure       1127
               ...  
wildpeak        8357
winter         13334
winterforce     2726
wrangler       16057
yokohama       15924
Length: 179, dtype: int64

In [ ]:
df_words.shape

In [ ]:
df_measures[df['Clicks'] == 1]

In [ ]:
df_measures['list_counts'].value_counts()

In [25]:
df_words[list(test.index)]

,accelera,achilles,advanta,advantage,adventure,aggressive,alenza,altimax,amazon,americus,...,weather,weight,westlake,wheel,whitewall,wildpeak,winter,winterforce,wrangler,yokohama
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355666,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1355667,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1355668,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1355669,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df_measures

,Search_term,Cost,Avg_cpc,Impressions,Clicks,Conversions,Conv_value,Conv_rate,Conv_value_cost,list_dimension,list_counts
0,285 60r20 sale,2.22,0.74,3,3,0.00,0.00,0.00,0.00,[285 60r20],1
1,325 45r22 nitto 420s,0.22,0.22,1,1,0.00,0.00,0.00,0.00,[325 45r22],1
4,285 75r16 at tires,21.69,0.39,263,56,0.63,438.10,1.10,20.20,[285 75r16],1
5,295 65r15,65.89,0.21,3098,314,0.50,106.26,0.16,1.61,[295 65r15],1
6,235 65 17 toyo tires open country,0.76,0.76,1,1,0.00,0.00,0.00,0.00,[235 65 17],1
...,...,...,...,...,...,...,...,...,...,...,...
2249697,atturo mud tires 265 65 16,0.34,0.34,3,1,0.00,0.00,0.00,0.00,[265 65 16],1
2249698,35 11.50 r20 bfg,0.68,0.68,1,1,0.00,0.00,0.00,0.00,[35 11.50 r20 b],1
2249699,maxxis tires razr 275 65r18 3 ply,0.52,0.26,2,2,0.00,0.00,0.00,0.00,[r 275 65r18 3 p],1
2249700,walmart tires for sale tire prices285 75 16,0.00,0.00,0,0,0.00,0.00,0.00,0.00,[285 75 16],1


In [ ]:
df_singles.head()

In [ ]:
df_singles['Conversions'].value_counts()